In [1]:
# YOLO para detecção de objetos em tempo real via webcam
from ultralytics import YOLO

# importa OpenCV para captura de vídeo e manipulação de imagens
import cv2

# carrega o modelo YOLOv11 nano pré-treinado
model = YOLO("yolo11n.pt")

# estancia da webcam (0 = câmera padrão)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Não foi possível abrir a webcam.")

# 3) loop principal
while True:
    # ret (tem video ou não), frame (imagem capturada)
    ret, frame = cap.read()  # lê um frame da webcam (BGR)
    # se não conseguiu ler, sai do loop
    if not ret:
        break

    # converte de BGR para RGB (o modelo usa RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # faz a inferência (um resultado para a imagem), conf é o limitador de confiança sendo 0.25 é o minimo
    results = model.predict(frame_rgb, conf=0.25, verbose=False)
    r = results[0]  # pega o resultado da primeira (única) imagem

    # extrai: coordenadas das caixas (x1,y1,x2,y2), confiança (score) e índice da classe (número).
    boxes = r.boxes.xyxy.cpu().numpy() if hasattr(r.boxes.xyxy, "cpu") else r.boxes.xyxy
    scores = (
        r.boxes.conf.cpu().numpy() if hasattr(r.boxes.conf, "cpu") else r.boxes.conf
    )
    class_ids = (
        r.boxes.cls.cpu().numpy().astype(int)
        if hasattr(r.boxes.cls, "cpu")
        else r.boxes.cls.astype(int)
    )

    # desenha cada detecção na imagem original (BGR)
    # para cada detecção, pegamos caixa, score e id da classe.
    for (x1, y1, x2, y2), s, cid in zip(boxes, scores, class_ids):
        # transforma coordenadas em inteiros (pixels).
        x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
        # converte o rótulo da classe e a confiança em texto
        label = f"{model.names[int(cid)]} {s:.2f}"
        # desenha a caixa e o rótulo na imagem
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # escreve o rótulo (classe + score) acima da caixa.
        cv2.putText(
            frame, label, (x1, y1 - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2
        )

    # mostra o frame com anotações e permite sair com 'q'
    cv2.imshow("YOLO - webcam (q para sair)", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# libera recursos
cap.release()
cv2.destroyAllWindows()